In [0]:
import requests
import base64
import json

In [0]:
def get_access_token():
        client_id = dbutils.secrets.get(scope="spotify_secrets",key="user")
        client_secret = dbutils.secrets.get(scope="spotify_secrets",key="client_secret")    

        # 1. Codificare Base64 pentru Client ID și Client Secret
        auth_string = f"{client_id}:{client_secret}"
        auth_bytes = auth_string.encode("utf-8")
        auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
        auth_url = 'https://accounts.spotify.com/api/token'

        headers = {
            "Authorization": f"Basic {auth_base64}",
            "Content-Type": "application/x-www-form-urlencoded"
        }

        data = {"grant_type": "client_credentials"}
        # 2. API request
        response = requests.post(auth_url, headers=headers, data=data)
        
        if response.status_code == 200:
            return response.json()['access_token']
        else:
            raise Exception("Nu s-a putut reîmprospăta token-ul.")
    

def get_authorization_header(self,token):
    return {"Authorization": f"Basic {token}"}

In [0]:
# --- Datele tale (presupunem că le ai deja) ---
token = get_access_token()
playlist_id = "5h1oEk4W9KVMHkOd8WWWlC"

headers = {"Authorization": f"Bearer {token}"}

# --- Construirea cererii inițiale ---
# Folosim 'fields' pentru a cere doar ce ne interesează și a face răspunsul mai curat
fields_query = "items(track(id,name,uri)),next"
initial_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields={fields_query}&limit=100"
# initial_url =  f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

print(f"Încep extragerea melodiilor din playlistul {playlist_id}...")

# --- Logica de paginare ---
all_playlist_items = []
current_url = initial_url

while current_url:
    try:
        response = requests.get(current_url, headers=headers)
        # Verifică dacă cererea a avut succes (cod 200 OK)
        response.raise_for_status() 
        data = response.json()
        # Adaugă melodiile din pagina curentă la lista totală
        if 'items' in data:
            all_playlist_items.extend(data['items'])
            print(f"Am extras {len(data['items'])} melodii. Total acum: {len(all_playlist_items)}.")
        # Pregătește următorul apel folosind link-ul de paginare
        current_url = data.get('next') 
    except requests.exceptions.HTTPError as err:
        print(f"A apărut o eroare HTTP: {err}")
        print(f"Răspuns de la server: {response.text}")
        break
    except Exception as e:
        print(f"A apărut o eroare neașteptată: {e}")
        break
print("\n--- Extragere finalizată! ---")
print(f"Numărul total de melodii în playlist: {len(all_playlist_items)}")


In [0]:
tracks_data = [item['track'] for item in all_playlist_items if item.get('track')]
bronze_playlist_df = spark.createDataFrame(tracks_data)


In [0]:
bronze_playlist_df.write.format("delta").mode("overwrite").saveAsTable("spotify_etl.bronze.bronze_playlist")